<a href="https://colab.research.google.com/github/PrzemyslawSarnacki/AirQualityPrediction/blob/master/PredictionNotebook.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a> 

In [ ]:
!pip install py-openaq

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import openaq
import warnings

warnings.simplefilter('ignore')

%matplotlib inline

# Set major seaborn asthetics
sns.set("notebook", style='ticks', font_scale=1.0)

# Increase the quality of inline plots
mpl.rcParams['figure.dpi']= 500


In [4]:
api = openaq.OpenAQ()
resp = api.cities(df=True, limit=10000)

# display the first 10 rows
resp.query("country == 'PL'")

,country,name,city,count,locations
2001,PL,Augustów,Augustów,88266,2
2002,PL,Belsk,Belsk,32105,1
2003,PL,Belsk Duży,Belsk Duży,87034,1
2004,PL,Biała,Biała,27573,1
2005,PL,Biała Podlaska,Biała Podlaska,106955,1
...,...,...,...,...,...
2171,PL,Zielonka,Zielonka,98239,1
2172,PL,Złoty Potok,Złoty Potok,133381,1
2173,PL,Żory,Żory,48437,1
2174,PL,Żyrardów,Żyrardów,45537,1


In [5]:
CITIES = ["Warszawa", "Kraków", "Poznań", "Katowice", "Białystok"]
PARAMETERS = ['pm25', 'pm10', 'no2', 'so2', 'o3', 'co']
LOCATIONS = ["Warszawa-Śródmieście", "Kraków-", "Poznań", "Katowice", "Białystok-Miejska", "WIOŚ Elbląg ul. Bażyńskiego"]

In [6]:
import unicodedata

def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text.lower().replace("ł","l")) if unicodedata.category(c) != 'Mn')


In [7]:
def check_params():
    return all(param in res.parameter.unique() for param in PARAMETERS) 

dfs = {}

for city in CITIES:
    print(city)
    dfs[strip_accents(city)] = pd.DataFrame()
    for page in range(1, 25):
        if (city == "Katowice" or city == "Poznań" ) and page > 15:
            continue
        if city == "Białystok" and page > 12:
            continue
        res = api.measurements(city=city, page=page, limit=10000, df=True)
        print(check_params())
        if check_params():
            dfs[strip_accents(city)] = dfs[strip_accents(city)].append(res)


Warszawa
True
True
True
True
True
True
True
True
True
True
Kraków
True
True
True
True
True
True
True
True
True
True
Poznań
True
True
True
True
True
True
True
True
True
True
Katowice
True
True
True
True
True
True
True
True
True
True
Białystok
True
True
True
True
True
True
True
True
True
False


In [8]:
dfs["bialystok"].head()

,location,parameter,value,unit,country,city,date.utc,coordinates.latitude,coordinates.longitude
date.local,,,,,,,,,
2020-12-22 16:00:00,Białystok-Warszawska,o3,22.00,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-22 15:00:00+00:00,53.129306,23.181744
2020-12-22 16:00:00,Białystok-Miejska,no2,10.10,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-22 15:00:00+00:00,53.126689,23.155869
2020-12-22 16:00:00,Białystok-Miejska,co,417.00,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-22 15:00:00+00:00,53.126689,23.155869
2020-12-22 16:00:00,Białystok-Miejska,bc,0.84,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-22 15:00:00+00:00,53.126689,23.155869
2020-12-22 16:00:00,Białystok-Warszawska,pm10,26.28,b'\xc2\xb5g/m\xc2\xb3',PL,Białystok,2020-12-22 15:00:00+00:00,53.129306,23.181744


In [9]:
organized_dfs = {}

for city in CITIES:    
    df_avg = pd.DataFrame()
    for param in PARAMETERS:
        df_avg[param] = dfs[strip_accents(city)].loc[dfs[strip_accents(city)]["parameter"] == param].resample('D').mean()["value"]
        organized_dfs[strip_accents(city)] = df_avg
        

In [ ]:
output = pd.concat([organized_dfs[strip_accents(city)] for city in CITIES], axis=1, keys=[strip_accents(city) for city in CITIES])

In [ ]:
output.to_csv("airq_data_1.csv")